<a href="https://colab.research.google.com/github/Ashonet/S-P100_ANALYSIS/blob/main/Stock_Cashflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

def get_stock_data(symbol, start_date, end_date):
    try:
        stock = yf.download(symbol, start=start_date, end=end_date)
        return stock
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return pd.DataFrame()

def get_sp100_companies():
    sp100_symbols = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')[2]
    sp100_symbols = sp100_symbols['Symbol'].tolist()
    return sp100_symbols

def get_free_cash_flow(symbol):
    try:
        ticker = yf.Ticker(symbol)
        cash_flow = ticker.cashflow
        fcf = cash_flow.loc['Free Cash Flow']
        fcf = fcf[fcf > 0]  # Only consider positive FCF values
        return fcf
    except Exception as e:
        print(f"Error fetching free cash flow data for {symbol}: {e}")
        return None

def predict_future_fcf(fcf, years=5, capped_growth_rate=0.2):
    try:
        historical_growth_rates = fcf.pct_change().dropna()
        average_growth_rate = historical_growth_rates.mean()
        average_growth_rate = min(average_growth_rate, capped_growth_rate)  # Cap the growth rate

        future_fcf = [fcf.iloc[-1]]
        for _ in range(years):
            future_fcf.append(future_fcf[-1] * (1 + average_growth_rate))

        return future_fcf
    except Exception as e:
        print(f"Error predicting future free cash flow: {e}")
        return None

def calculate_intrinsic_value(future_fcf, fcf_yield):
    try:
        # Future intrinsic value is calculated as future FCF divided by FCF yield
        intrinsic_value = future_fcf[-1] / fcf_yield
        return intrinsic_value
    except Exception as e:
        print(f"Error calculating intrinsic value: {e}")
        return None

def get_outstanding_shares(symbol):
    try:
        ticker = yf.Ticker(symbol)
        shares = ticker.info.get('sharesOutstanding', None)
        return shares
    except Exception as e:
        print(f"Error fetching outstanding shares for {symbol}: {e}")
        return None

def get_market_cap(symbol):
    try:
        ticker = yf.Ticker(symbol)
        market_cap = ticker.info.get('marketCap', None)
        return market_cap
    except Exception as e:
        print(f"Error fetching market cap for {symbol}: {e}")
        return None

def estimate_stock_price(symbol, years=5, capped_growth_rate=0.2):
    fcf = get_free_cash_flow(symbol)
    if fcf is None:
        return None

    future_fcf = predict_future_fcf(fcf, years, capped_growth_rate)
    if future_fcf is None:
        return None

    market_cap = get_market_cap(symbol)
    if market_cap is None:
        return None

    fcf_yield = fcf.iloc[-1] / market_cap
    intrinsic_value = calculate_intrinsic_value(future_fcf, fcf_yield)
    if intrinsic_value is None:
        return None

    outstanding_shares = get_outstanding_shares(symbol)
    if outstanding_shares is None:
        return None

    stock_price = intrinsic_value / outstanding_shares
    return stock_price

def main():
    sp100_symbols = get_sp100_companies()

    for symbol in sp100_symbols:
        stock_price = estimate_stock_price(symbol)
        if stock_price is not None:
            # Fetch current share price
            ticker = yf.Ticker(symbol)
            current_price = ticker.history(period="1d")['Close'][0]
            print(f"Predicted stock price for {symbol}: ${stock_price:.2f}, Current share price: ${current_price:.2f}")

            strong_upper_bound = current_price * 1.3
            upper_bound = current_price * 1.1
            lower_bound = current_price * 0.9
            strong_lower_bound = current_price * 0.7

            if stock_price > strong_upper_bound:
                print(f"{symbol}: Strong Buy")
            elif stock_price > upper_bound:
                print(f"{symbol}: Buy")
            elif stock_price < strong_lower_bound:
                print(f"{symbol}: Strong Sell")
            elif stock_price < lower_bound:
                print(f"{symbol}:  Sell")
            elif lower_bound <= stock_price <= upper_bound:
                print(f"{symbol}: Hold")
            else:
                print(f"No data available for {symbol}")
        else:
            print(f"Could not predict stock price for {symbol}")

# Call the main function to execute the analysis
if __name__ == "__main__":
    main()


Predicted stock price for AAPL: $121.21, Current share price: $189.87
AAPL: Strong Sell
Predicted stock price for ABBV: $111.58, Current share price: $166.42
ABBV: Strong Sell
Predicted stock price for ABT: $170.87, Current share price: $104.09
ABT: Strong Buy
Predicted stock price for ACN: $230.60, Current share price: $303.59
ACN:  Sell
Predicted stock price for ADBE: $322.79, Current share price: $483.43
ADBE: Strong Sell
Predicted stock price for AIG: $22.97, Current share price: $80.54
AIG: Strong Sell
Predicted stock price for AMD: $409.23, Current share price: $164.47
AMD: Strong Buy
Predicted stock price for AMGN: $524.73, Current share price: $312.47
AMGN: Strong Buy
Predicted stock price for AMT: $332.74, Current share price: $194.51
AMT: Strong Buy
Predicted stock price for AMZN: $62.31, Current share price: $184.70
AMZN: Strong Sell
Predicted stock price for AVGO: $698.61, Current share price: $1395.29
AVGO: Strong Sell
Predicted stock price for AXP: $43.44, Current share p